# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [1]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

Cloning into 'SQuAD-explorer'...


remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 12.47 MiB/s, done.
Resolving deltas: 100% (3563/3563), done.


In [2]:
import json

In [3]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [4]:
type(train_data)

dict

In [5]:
list(train_data.keys())

['data', 'version']

In [6]:
type(train_data["data"])

list

In [7]:
len(train_data["data"])

442

In [8]:
type(train_data["data"][0])

dict

In [9]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [10]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [11]:
len(train_data["data"][0]["paragraphs"])

55

In [12]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [13]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [14]:
# YOUR CHANGES HERE

# Part 2: build parsed.tsv
import pandas as pd

rows = []

# train_data was already loaded above
for doc in train_data["data"]:
    title = doc["title"]
    for idx, para in enumerate(doc["paragraphs"]):
        context = para["context"]
        rows.append(
            {
                "document_title": title,
                "paragraph_index": idx,          # zero-indexed
                "paragraph_context": context,
            }
        )

parsed_df = pd.DataFrame(rows)
parsed_df.to_csv("parsed.tsv", sep="\t", index=False)

parsed_df.head()


,document_title,paragraph_index,paragraph_context
0,University_of_Notre_Dame,0,"Architecturally, the school has a Catholic cha..."
1,University_of_Notre_Dame,1,"As at most other universities, Notre Dame's st..."
2,University_of_Notre_Dame,2,The university is the major seat of the Congre...
3,University_of_Notre_Dame,3,The College of Engineering was established in ...
4,University_of_Notre_Dame,4,All of Notre Dame's undergraduate students are...


Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [15]:
# YOUR CHANGES HERE

# Part 3 (1/2): create 1024-dim paragraph vectors using TF-IDF
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Read the parsed paragraphs
parsed_df = pd.read_csv("parsed.tsv", sep="\t")

# TF-IDF with vocabulary size 1024
tfidf = TfidfVectorizer(max_features=1024)
paragraph_matrix = tfidf.fit_transform(parsed_df["paragraph_context"])

paragraph_matrix.shape


(18896, 1024)

Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [16]:
# YOUR CHANGES HERE

# Part 3 (2/2): save paragraph vectors to paragraph-vectors.tsv.gz
import json
import numpy as np

# Convert sparse matrix to dense (n_paragraphs x 1024)
paragraph_dense = paragraph_matrix.toarray().astype(float)

paragraph_vector_json = [
    json.dumps(vec.tolist()) for vec in paragraph_dense
]

paragraph_vectors_df = pd.DataFrame(
    {
        "document_title": parsed_df["document_title"],
        "paragraph_index": parsed_df["paragraph_index"],
        "paragraph_vector_json": paragraph_vector_json,
    }
)

paragraph_vectors_df.to_csv(
    "paragraph-vectors.tsv.gz", sep="\t", index=False
)

paragraph_vectors_df.head()


,document_title,paragraph_index,paragraph_vector_json
0,University_of_Notre_Dame,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,University_of_Notre_Dame,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,University_of_Notre_Dame,2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,University_of_Notre_Dame,3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,University_of_Notre_Dame,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [17]:
# YOUR CHANGES HERE

# Part 4 (1/2): encode question vectors with the SAME tfidf as paragraphs
import pandas as pd

questions_df = pd.read_csv("questions.tsv", sep="\t")

# Use the SAME fitted tfidf object from Part 3
question_matrix = tfidf.transform(questions_df["question"])

question_matrix.shape


(100, 1024)

Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [18]:
# YOUR CHANGES HERE

# Part 4 (2/2): save question vectors to question-vectors.tsv
import json
import numpy as np

question_dense = question_matrix.toarray().astype(float)

question_vector_json = [
    json.dumps(vec.tolist()) for vec in question_dense
]

question_vectors_df = pd.DataFrame(
    {
        "question_id": questions_df["question_id"],
        "question_vector_json": question_vector_json,
    }
)

question_vectors_df.to_csv(
    "question-vectors.tsv", sep="\t", index=False
)

question_vectors_df.head()


,question_id,question_vector_json
0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,7,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,10,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,13,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [19]:
# YOUR CHANGES HERE
# Part 5 (1/2): find top-5 nearest paragraphs for each question
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

# We assume parsed_df, paragraph_matrix, questions_df, question_matrix
# are already in memory from Parts 3 and 4.

n_neighbors = 5
nn = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
nn.fit(paragraph_matrix)

distances, indices = nn.kneighbors(question_matrix)

rows = []
for q_idx, (q_id, neigh_indices) in enumerate(
    zip(questions_df["question_id"], indices)
):
    for rank, p_idx in enumerate(neigh_indices, start=1):
        rows.append(
            {
                "question_id": q_id,
                "question_rank": rank,
                "document_title": parsed_df.loc[p_idx, "document_title"],
                "paragraph_index": int(parsed_df.loc[p_idx, "paragraph_index"]),
            }
        )

matches_df = pd.DataFrame(rows)
matches_df.head()


,question_id,question_rank,document_title,paragraph_index
0,1,1,Tuvalu,49
1,1,2,Genome,8
2,1,3,Rajasthan,9
3,1,4,Bill_%26_Melinda_Gates_Foundation,6
4,1,5,Tibet,10


Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [20]:
# YOUR CHANGES HERE

# Part 5 (2/2): save matches to question-matches.tsv
matches_df.to_csv("question-matches.tsv", sep="\t", index=False)
matches_df.head()


,question_id,question_rank,document_title,paragraph_index
0,1,1,Tuvalu,49
1,1,2,Genome,8
2,1,3,Rajasthan,9
3,1,4,Bill_%26_Melinda_Gates_Foundation,6
4,1,5,Tibet,10


Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


In [22]:
# Part 6: build worst-paragraphs.tsv AFTER manual inspection
import pandas as pd

# TODO: replace the example rows with your actual choices
worst_rows = [
    # Example format (replace with real values):
    # {"question_id": 1, "document_title": "Some_Title", "paragraph_index": 123},
    # {"question_id": 4, "document_title": "Another_Title", "paragraph_index": 45},
]

worst_df = pd.DataFrame(worst_rows)
worst_df.to_csv("worst-paragraphs.tsv", sep="\t", index=False)
worst_df


""


Submit "worst-paragraphs.tsv" in Gradescope.

In [21]:
# Part 6: inspect matches for first 5 questions
import pandas as pd

# Make sure these are in memory (from earlier cells); if not, reload:
# parsed_df = pd.read_csv("parsed.tsv", sep="\t")
# matches_df = pd.read_csv("question-matches.tsv", sep="\t")
# questions_df = pd.read_csv("questions.tsv", sep="\t")

first5_ids = sorted(questions_df["question_id"].unique())[:5]

for q_id in first5_ids:
    q_text = questions_df.loc[
        questions_df["question_id"] == q_id, "question"
    ].iloc[0]
    print("=" * 100)
    print(f"QUESTION {q_id}: {q_text}\n")

    sub = matches_df[matches_df["question_id"] == q_id].sort_values(
        "question_rank"
    )

    for _, row in sub.iterrows():
        para = parsed_df[
            (parsed_df["document_title"] == row["document_title"])
            & (parsed_df["paragraph_index"] == row["paragraph_index"])
        ]["paragraph_context"].iloc[0]

        print(
            f"  rank {row['question_rank']} | "
            f"doc={row['document_title']} | para_index={row['paragraph_index']}"
        )
        print(para)
        print("-" * 100)


QUESTION 1: What was the goal of the abuse of region project?

  rank 1 | doc=Tuvalu | para_index=49
The eastern shoreline of Funafuti Lagoon was modified during World War II when the airfield (what is now Funafuti International Airport) was constructed. The coral base of the atoll was used as fill to create the runway. The resulting borrow pits impacted the fresh-water aquifer. In the low areas of Funafuti the sea water can be seen bubbling up through the porous coral rock to form pools with each high tide. Since 1994 a project has been in development to assess the environmental impact of transporting sand from the lagoon to fill all the borrow pits and low-lying areas on Fongafale. In 2014 the Tuvalu Borrow Pits Remediation (BPR) project was approved in order to fill 10 borrow pits, leaving Tafua Pond, which is a natural pond. The New Zealand Government funded the BPR project. The project was carried out in 2015 with 365,000 sqm of sand being dredged from the lagoon to fill the holes

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [24]:
import pandas as pd

ack_data = [{
    "discussions": "none",
    "extra_libraries": "none",
    "generative_ai": (
        "Used OpenAI ChatGPT (GPT-5 Thinking) to help design and structure code "
        "for the DX704 Week 10 project; I reviewed, ran, and adjusted all code myself."
    ),
}]

ack_df = pd.DataFrame(ack_data)
ack_df.to_csv("acknowledgements.tsv", sep="\t", index=False)
ack_df


,discussions,extra_libraries,generative_ai
0,none,none,Used OpenAI ChatGPT (GPT-5 Thinking) to help d...
